[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PacktPublishing/Modern-Computer-Vision-with-PyTorch-2E/blob/main/Chapter10/action_recognition.ipynb)

In [ ]:
%%capture
# This will take approx 3 min
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install -U "openmim==0.3.9"
!mim install -U "mmengine==0.10.4"
!mim install "mmcv==2.2.0"
!git clone https://github.com/sizhky/mmaction2.git -b main
%cd mmaction2
%pip install -e .
%pip install -r requirements/optional.txt
%pip install "timm==0.9.16"
%pip install "torch-snippets==0.528" lovely-tensors

In [ ]:
# Check Pytorch installation
%reload_ext autoreload
%autoreload 2
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
# 2.2.1+cu121 True

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)
# 1.2.0

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
# GCC 9.3

print(get_compiler_version())

# Check MMEngine installation
from mmengine.utils.dl_utils import collect_env
print(collect_env())
_ = """
OrderedDict([
    ('sys.platform', 'linux'),
    ('Python', '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'),
    ('CUDA available', True),
    ('MUSA available', False),
    ('numpy_random_seed', 2147483648),
    ('GPU 0', 'Tesla T4'),
    ('CUDA_HOME', '/usr/local/cuda'),
    ('NVCC', 'Cuda compilation tools, release 12.2, V12.2.140'),
    ('GCC', 'x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0'),
    ('PyTorch', '2.2.1+cu121'),
    ('PyTorch compiling details', '''
        PyTorch built with:
            - GCC 9.3
            - C++ Version: 201703
            - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
            - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
            - OpenMP 201511 (a.k.a. OpenMP 4.5)
            - LAPACK is enabled (usually provided by MKL)
            - NNPACK is enabled
            - CPU capability usage: AVX2
            - CUDA Runtime 12.1
            - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
            - CuDNN 8.9.2
            - Magma 2.6.1
            - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINETO -DLIBKINETO_NOROCTRACER -DUSE_FBGEMM -DUSE_QNNPACK -DUSE_PYTORCH_QNNPACK -DUSE_XNNPACK -DSYMBOLICATE_MOBILE_DEBUG_HANDLE -O2 -fPIC -Wall -Wextra -Werror=return-type -Werror=non-virtual-dtor -Werror=bool-operation -Wnarrowing -Wno-missing-field-initializers -Wno-type-limits -Wno-array-bounds -Wno-unknown-pragmas -Wno-unused-parameter -Wno-unused-function -Wno-unused-result -Wno-strict-overflow -Wno-strict-aliasing -Wno-stringop-overflow -Wsuggest-override -Wno-psabi -Wno-error=pedantic -Wno-error=old-style-cast -Wno-missing-braces -fdiagnostics-color=always -faligned-new -Wno-unused-but-set-variable -Wno-maybe-uninitialized -fno-math-errno -fno-trapping-math -Werror=format -Wno-stringop-overflow, LAPACK_INFO=mkl, PERF_WITH_AVX=1, PERF_WITH_AVX2=1, PERF_WITH_AVX512=1, TORCH_VERSION=2.2.1, USE_CUDA=ON, USE_CUDNN=ON, USE_EXCEPTION_PTR=1, USE_GFLAGS=OFF, USE_GLOG=OFF, USE_MKL=ON, USE_MKLDNN=ON, USE_MPI=OFF, USE_NCCL=1, USE_NNPACK=ON, USE_OPENMP=ON, USE_ROCM=OFF, USE_ROCM_KERNEL_ASSERT=OFF,
        '''
    )
    ('TorchVision', '0.17.1+cu121'),
    ('OpenCV', '4.8.0'),
    ('MMEngine', '0.10.4')
])
"""

2.2.1+cu121 True
1.2.0
12.1
GCC 9.3
OrderedDict([('sys.platform', 'linux'), ('Python', '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]'), ('CUDA available', True), ('MUSA available', False), ('numpy_random_seed', 2147483648), ('GPU 0', 'Tesla T4'), ('CUDA_HOME', '/usr/local/cuda'), ('NVCC', 'Cuda compilation tools, release 12.2, V12.2.140'), ('GCC', 'x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0'), ('PyTorch', '2.2.1+cu121'), ('PyTorch compiling details', 'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201703\n  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX512\n  - CUDA Runtime 12.1\n  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,cod

## Perform inference with a MMAction2 recognizer
MMAction2 already provides high level APIs to do inference and training.

In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

--2024-04-25 17:35:47--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.20.218, 47.246.20.226, 47.246.20.227, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.20.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97579339 (93M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’

checkpoints/tsn_r50 100%[===================>]  93.06M  10.5MB/s    in 9.5s    

2024-04-25 17:35:57 (9.77 MB/s) - ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’ saved [97579339/97579339]



In [ ]:
from torch_snippets import *
from builtins import print

from mmaction.apis import inference_recognizer, init_recognizer
from mmengine import Config


# Choose to use a config and initialize the recognizer
config = 'configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py'
config = Config.fromfile(config)
# Setup a checkpoint file to load
checkpoint = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

Loads checkpoint by local backend from path: checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth


In [ ]:
model

Recognizer2D(
  (data_preprocessor): ActionDataPreprocessor()
  (backbone): ResNet(
    (conv1): ConvModule(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): ConvModule(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        

In [ ]:
# Use the recognizer to do inference
from operator import itemgetter

def predict():
    video = 'demo/demo.mp4'
    label = 'tools/data/kinetics/label_map_k400.txt'
    results = inference_recognizer(model, video)

    pred_scores = results.pred_score.tolist()
    score_tuples = tuple(zip(range(len(pred_scores)), pred_scores))
    score_sorted = sorted(score_tuples, key=itemgetter(1), reverse=True)
    top5_label = score_sorted[:5]

    labels = open(label).readlines()
    labels = [x.strip() for x in labels]
    results = [(labels[k[0]], k[1]) for k in top5_label]
    return results

from torch_snippets.trainer.hooks import print_module_ios_for
with print_module_ios_for(model, required_children=['backbone','cls_head.consensus', 'cls_head.avg_pool', 'cls_head.dropout']) as _:
    results = predict()


04/25 17:36:07 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
04/25 17:36:07 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.


══════════════════════════════════════════════════════════════════

Module Name: ResNet. Child Name: backbone
Input Kwargs: 

Input Args:
  1 - tensor[250, 3, 224, 224] n=37632000 (0.1Gb) x∈[-2.118, 2.640] μ=-0.479 σ=1.226 cuda:0 - ID:#6cb02b

Outputs: 
  1 - tensor[250, 2048, 7, 7] n=25088000 (96Mb) x∈[0., 15.620] μ=0.438 σ=0.798 cuda:0 - ID:#e6d1d1



══════════════════════════════════════════════════════════════════

══════════════════════════════════════════════════════════════════

...

Module Name: AvgConsensus. Child Name: cls_head.consensus
Input Kwargs: 

Input Args:
  1 - tensor[1, 250, 2048, 1, 1] n=512000 (2.0Mb) x∈[0., 5.002] μ=0.438 σ=0.497 cuda:0 - ID:#0ac947

Outputs: 
  1 - tensor[1, 1, 2048, 1, 1] 8Kb x∈[0.020, 2.895] μ=0.438 σ=0.414 cuda:0 - ID:#08fb2f



══════════════════════════════════════════════════════════════════

══════════════════════════════════════════════════════════════════

Module Name: Dropout. Child Name: cls_head.dropout
Input Kwargs: 

Input Args:
  1 - tensor[1, 2048, 1, 1] 8Kb x∈[0.020, 2.895] μ=0.438 σ=0.414 cuda:0 - ID:#08fb2f

Outputs: 
  1 - tensor[1, 2048, 1, 1] 8Kb x∈[0.020, 2.895] μ=0.438 σ=0.414 cuda:0 - ID:#08fb2f



══════════════════════════════════════════════════════════════════

In [ ]:
print('The top-5 labels with corresponding scores are:')
for result in results:
    print(f'{result[0]}: ', result[1])

The top-5 labels with corresponding scores are:
arm wrestling:  1.0
rock scissors paper:  6.434453414527752e-09
shaking hands:  2.7599860175087088e-09
clapping:  1.345463851443185e-09
massaging feet:  5.555100823784187e-10


## Train a recognizer on customized dataset

To train a new recognizer, there are usually three things to do:
1. Support a new dataset
2. Modify the config
3. Train a new recognizer

### Support a new dataset

In this tutorial, we gives an example to convert the data into the format of existing datasets. Other methods and more advanced usages can be found in the [doc](/docs/tutorials/new_dataset.md)

Firstly, let's download a tiny dataset obtained from [Kinetics-400](https://deepmind.com/research/open-source/open-source-datasets/kinetics/). We select 30 videos with their labels as train dataset and 10 videos with their labels as test dataset.

In [ ]:
# download, decompress the data
!rm kinetics400_tiny.zip*
!rm -rf kinetics400_tiny
!wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip > /dev/null

rm: cannot remove 'kinetics400_tiny.zip*': No such file or directory
--2024-04-25 17:36:11--  https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.20.218, 47.246.20.226, 47.246.20.227, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.20.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18308682 (17M) [application/zip]
Saving to: ‘kinetics400_tiny.zip’

kinetics400_tiny.zi 100%[===================>]  17.46M  7.67MB/s    in 2.3s    

2024-04-25 17:36:14 (7.67 MB/s) - ‘kinetics400_tiny.zip’ saved [18308682/18308682]



In [ ]:
Glob('kinetics400_tiny/*/*')

[» kinetics400_tiny/val/SU_x2LQqSLs.mp4,
 » kinetics400_tiny/val/AQrbRSnRt8M.mp4,
 » kinetics400_tiny/val/y5Iu7XkTqV0.mp4,
 » kinetics400_tiny/val/b6Q_b7vgc7Q.mp4,
 » kinetics400_tiny/val/jqRrH30V0k4.mp4,
 » kinetics400_tiny/val/ddvJ6-faICE.mp4,
 » kinetics400_tiny/val/0pVGiAU6XEA.mp4,
 » kinetics400_tiny/val/ik4BW3-SCts.mp4,
 » kinetics400_tiny/val/u4Rm6srmIS8.mp4,
 » kinetics400_tiny/val/IcLztCtvhb8.mp4,
 » kinetics400_tiny/train/oXy-e_P_cAI.mp4,
 » kinetics400_tiny/train/ZQV4U2KQ370.mp4,
 » kinetics400_tiny/train/xGY2dP0YUjA.mp4,
 » kinetics400_tiny/train/R8HXQkdgKWA.mp4,
 » kinetics400_tiny/train/phDqGd0NKoo.mp4,
 » kinetics400_tiny/train/yLC9CtWU5ws.mp4,
 » kinetics400_tiny/train/LvcFDgCAXQs.mp4,
 » kinetics400_tiny/train/FMlSTTpN3VY.mp4,
 » kinetics400_tiny/train/34XczvTaRiI.mp4,
 » kinetics400_tiny/train/TkkZPZHbAKA.mp4,
 » kinetics400_tiny/train/iRuyZSKhHRg.mp4,
 » kinetics400_tiny/train/oMrZaozOvdQ.mp4,
 » kinetics400_tiny/train/DbX8mPslRXg.mp4,
 » kinetics400_tiny/train/RqnKt

In [ ]:
# After downloading the data, we need to check the annotation format
!cat kinetics400_tiny/kinetics_tiny_train_video.txt

D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0


According to the format defined in [`VideoDataset`](./datasets/video_dataset.py), each line indicates a sample video with the filepath and label, which are split with a whitespace.

### Modify the config

In the next step, we need to modify the config for the training.
To accelerate the process, we finetune a recognizer using a pre-trained recognizer.

In [ ]:
from mmengine import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py')

Given a config that trains a TSN model on kinetics400-full dataset, we need to modify some values to use it for training TSN on Kinetics400-tiny dataset.


In [ ]:
from mmengine.runner import set_random_seed

# Modify dataset type and path
cfg.data_root = 'kinetics400_tiny/train/'
cfg.data_root_val = 'kinetics400_tiny/val/'
cfg.ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'


cfg.test_dataloader.dataset.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.test_dataloader.dataset.data_prefix.video = 'kinetics400_tiny/val/'

cfg.train_dataloader.dataset.ann_file = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.train_dataloader.dataset.data_prefix.video = 'kinetics400_tiny/train/'

cfg.val_dataloader.dataset.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.val_dataloader.dataset.data_prefix.video  = 'kinetics400_tiny/val/'


# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './output'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.train_dataloader.batch_size = cfg.train_dataloader.batch_size // 16
cfg.val_dataloader.batch_size = cfg.val_dataloader.batch_size // 16
cfg.optim_wrapper.optimizer.lr = cfg.optim_wrapper.optimizer.lr / 8 / 16
cfg.train_cfg.max_epochs = 10

cfg.train_dataloader.num_workers = 2
cfg.val_dataloader.num_workers = 2
cfg.test_dataloader.num_workers = 2

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
auto_scale_lr = dict(base_batch_size=256, enable=False)
data_root = 'kinetics400_tiny/train/'
data_root_val = 'kinetics400_tiny/val/'
dataset_type = 'VideoDataset'
default_hooks = dict(
    checkpoint=dict(
        interval=3, max_keep_ckpts=3, save_best='auto', type='CheckpointHook'),
    logger=dict(ignore_last=False, interval=20, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    runtime_info=dict(type='RuntimeInfoHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    sync_buffers=dict(type='SyncBuffersHook'),
    timer=dict(type='IterTimerHook'))
default_scope = 'mmaction'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
file_client_args = dict(io_backend='disk')
load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [ ]:
import os.path as osp
import mmengine
from mmengine.runner import Runner

# Create work_dir
mmengine.mkdir_or_exist(osp.abspath(cfg.work_dir))

# build the runner from config
runner = Runner.from_cfg(cfg)

# start training
trained_model = runner.train()

04/25 17:37:27 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 978769941
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.2, V12.2.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.2.1+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=s

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


04/25 17:37:29 - mmengine - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.weight', 'fc.bias'}
Loads checkpoint by local backend from path: ./checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
04/25 17:37:30 - mmengine - INFO - Load checkpoint from ./checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
04/25 17:37:30 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
04/25 17:37:30 - mmengine - INFO - Checkpoints will be saved to /content/mmaction2/output.


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


04/25 17:37:34 - mmengine - INFO - Exp name: tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb_20240425_173727
04/25 17:37:34 - mmengine - INFO - Epoch(train)  [1][15/15]  lr: 7.8125e-05  eta: 0:00:40  time: 0.3033  data_time: 0.1071  memory: 2917  grad_norm: 12.8847  loss: 0.7189  top1_acc: 0.5000  top5_acc: 1.0000  loss_cls: 0.7189
04/25 17:37:35 - mmengine - INFO - Epoch(val) [1][5/5]    acc/top1: 0.6000  acc/top5: 1.0000  acc/mean1: 0.6000  data_time: 0.2115  time: 0.2445
04/25 17:37:36 - mmengine - INFO - The best checkpoint with 0.6000 acc/top1 at 1 epoch is saved to best_acc_top1_epoch_1.pth.
04/25 17:37:39 - mmengine - INFO - Exp name: tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb_20240425_173727
04/25 17:37:39 - mmengine - INFO - Epoch(train)  [2][15/15]  lr: 7.8125e-05  eta: 0:00:30  time: 0.1921  data_time: 0.1034  memory: 961  grad_norm: 12.6078  loss: 0.7031  top1_acc: 1.0000  top5_acc: 1.0000  loss_cls: 0.7031
04/25 17:37:40 - mmengine - INFO - E

In [ ]:
runner.test()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


04/25 17:38:32 - mmengine - INFO - Epoch(test) [10/10]    acc/top1: 1.0000  acc/top5: 1.0000  acc/mean1: 1.0000  data_time: 0.0684  time: 0.8032


{'acc/top1': 1.0, 'acc/top5': 1.0, 'acc/mean1': 1.0}